### Implement gene knockout function

This notebook identifies the KO reactions using metadata as well as gene association objects in the model's reaction objects 

In [142]:
from Compass.compass import models, compass, utils
import importlib
import pandas as pd
import os
import cplex
import re
import numpy as np

In [143]:
# Making changes in the local git repo - reload the .py file 
importlib.reload(models)
importlib.reload(models.importSBML3)
importlib.reload(models.geneSymbols)
importlib.reload(models.importCommon)

importlib.reload(compass)

<module 'Compass.compass.compass' from '/home/dzhang/Compass_Integration/Compass/compass/compass/__init__.py'>

In [3]:
# Load in MD file 
metadata = pd.read_csv('HumanGEM_MD/HG_rxn_md_v115.csv')
metadata = metadata.set_index("ID")
metadata

,NAME,EQUATION,EC-NUMBER,GENE ASSOCIATION,GENE NAME ASSOCIATION,SUBSYSTEM,MIRIAM,CONFIDENCE SCORE,rxnKEGGID,rxnBiGGID,rxnRecon3DID,rxnMetaNetXID,rxnRheaID,rxnRheaMasterID
ID,,,,,,,,,,,,,,
MAR03905,ethanol:NAD+ oxidoreductase,ethanol[c] + NAD+[c] => acetaldehyde[c] + H+[c...,1.1.1.1;1.1.1.71,ENSG00000147576 or ENSG00000172955 or ENSG0000...,"[['ADHFE1'], ['ADH6'], ['ZADH2'], ['ADH1A'], [...",Glycolysis / Gluconeogenesis,kegg.reaction/R00754;bigg.reaction/ALCD2x;vmhr...,0,R00754,ALCD2x,ALCD2if,MNXR95725,RHEA:25291,RHEA:25290
MAR03907,Ethanol:NADP+ oxidoreductase,ethanol[c] + NADP+[c] => acetaldehyde[c] + H+[...,1.1.1.2,ENSG00000117448,[['AKR1A1']],Glycolysis / Gluconeogenesis,kegg.reaction/R00746;bigg.reaction/ALCD2y;vmhr...,0,R00746,ALCD2y,ALCD2yf,MNXR95726,NaN,NaN
MAR04097,Acetate:CoA ligase (AMP-forming),acetate[c] + ATP[c] + CoA[c] => acetyl-CoA[c] ...,6.2.1.1,ENSG00000131069,[['ACSS2']],Glycolysis / Gluconeogenesis,kegg.reaction/R00235;bigg.reaction/ACS;vmhreac...,0,R00235,ACS,ACS,MNXR95413,RHEA:23177,RHEA:23176
MAR04099,Acetate:CoA ligase (AMP-forming),acetate[m] + ATP[m] + CoA[m] => acetyl-CoA[m] ...,6.2.1.1,ENSG00000111058 or ENSG00000154930,"[['ACSS3'], ['ACSS1']]",Glycolysis / Gluconeogenesis,kegg.reaction/R00235;bigg.reaction/ACSm;vmhrea...,0,R00235,ACSm,ACSm,MNXR95413,RHEA:23177,RHEA:23176
MAR04108,acetyl adenylate:CoA acetyltransferase,acetyl adenylate[c] + CoA[c] => acetyl-CoA[c] ...,6.2.1.1,ENSG00000131069,[['ACSS2']],Glycolysis / Gluconeogenesis,kegg.reaction/R00236;vmhreaction/r0068;metanet...,0,R00236,NaN,r0068,MNXR105304,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MAR20066,MAR20066,2 H2O2[x] => 2 H2O[x] + O2[x],1.11.1.6; 1.11.1.21,ENSG00000121691,[['CAT']],Glyoxylate and dicarboxylate metabolism,kegg.reaction/R00009;bigg.reaction/CAT;bigg.re...,0,R00009,CAT;CATm;CATp;CATpp;CATr;R_CAT;R_CATm;R_CATp;R...,NaN,MNXR96455,RHEA:20309,NaN
MAR20067,MAR20067,NAD+[c] + all-trans-4-hydroxyretinol[c] => H+[...,1.1.1.1,ENSG00000196616,[['ADH1B']],Fatty acid metabolism,metanetx.reaction/MNXR133799;rhea/55936;sbo/SB...,0,NaN,NaN,NaN,MNXR133799,RHEA:55936,NaN
MAR20068,MAR20068,4-oxoretinol[c] + NAD+[c] => H+[c] + NADH[c] +...,NaN,ENSG00000196616,[['ADH1B']],Fatty acid degradation,metanetx.reaction/MNXR171534;rhea/60632;sbo/SB...,0,NaN,NaN,NaN,MNXR171534,RHEA:60632,NaN


In [5]:
# Iterate through every rxn in MD file and check if KO is possible given provided genes 
# Set fluxes to 0 

# Every OR association has to be KO 

gene_list = ['ELOVL1']

KO_rxn_list = [] 

for rxn in metadata.index:
    if isinstance(metadata.loc[rxn, "GENE NAME ASSOCIATION"], str): 
        for sublist in metadata.loc[rxn, "GENE NAME ASSOCIATION"].split("],"):
            # Test each sublist 
            rxn_KO = False 
            genes = re.findall(r"'\w*'", sublist)
            for gene in genes:
                # Need only one gene in sublist 
                if gene[1:-1] in gene_list:
                    rxn_KO = True 
                    break
            # Test if the sublist can be KO 
            if not rxn_KO:
                break 
        # Only will append if all the sublists can be KO 
        if rxn_KO:
            KO_rxn_list.append(rxn) 

In [6]:
KO_rxn_list

['MAR02521',
 'MAR03394',
 'MAR03395',
 'MAR03401',
 'MAR03402',
 'MAR03403',
 'MAR03405',
 'MAR03410',
 'MAR03412',
 'MAR03417',
 'MAR03418',
 'MAR03438',
 'MAR03442',
 'MAR03451',
 'MAR03465',
 'MAR03474',
 'MAR03479',
 'MAR03487',
 'MAR03490',
 'MAR03492',
 'MAR03495',
 'MAR03496',
 'MAR03497',
 'MAR03499',
 'MAR03502',
 'MAR03504',
 'MAR03507',
 'MAR03518',
 'MAR03535',
 'MAR03607',
 'MAR03608',
 'MAR03609',
 'MAR03610',
 'MAR03611',
 'MAR03612',
 'MAR03615',
 'MAR03616',
 'MAR03617',
 'MAR03621',
 'MAR03624',
 'MAR03749']

In [43]:
# Test if gene name exists in the model before identifying KO reactions 
gene_list = ['ACSL5']

rxns = set() 

for rxn in metadata.index:
    if isinstance(metadata.loc[rxn, "GENE NAME ASSOCIATION"], str): 
        for sublist in metadata.loc[rxn, "GENE NAME ASSOCIATION"].split("],"):
            # Test each sublist 
            rxn_KO = False 
            genes = re.findall(r"'\w*'", sublist)
            for gene in genes:
                # Need only one gene in sublist 
                if gene[1:-1] in gene_list:
                    rxn_KO = True 
                    rxns.add(rxn)
                    break
            if rxn_KO:
                break 

In [44]:
rxns

{'MAR00188',
 'MAR00192',
 'MAR00196',
 'MAR00200',
 'MAR00204',
 'MAR00209',
 'MAR00213',
 'MAR00217',
 'MAR00226',
 'MAR00233',
 'MAR00237',
 'MAR00241',
 'MAR00245',
 'MAR00249',
 'MAR00255',
 'MAR00259',
 'MAR00263',
 'MAR00267',
 'MAR00271',
 'MAR00275',
 'MAR00279',
 'MAR00283',
 'MAR00289',
 'MAR00293',
 'MAR00297',
 'MAR00301',
 'MAR00305',
 'MAR00309',
 'MAR00313',
 'MAR00319',
 'MAR00323',
 'MAR00327',
 'MAR00331',
 'MAR00337',
 'MAR00341',
 'MAR00345',
 'MAR00349',
 'MAR00353',
 'MAR00357',
 'MAR00361',
 'MAR00365',
 'MAR00369',
 'MAR00373',
 'MAR00377',
 'MAR00381',
 'MAR00385',
 'MAR00389',
 'MAR00393',
 'MAR00397',
 'MAR00401',
 'MAR00405',
 'MAR00409',
 'MAR00413',
 'MAR00417',
 'MAR00421',
 'MAR00425',
 'MAR00429',
 'MAR00433',
 'MAR00437',
 'MAR01157',
 'MAR01158',
 'MAR01159',
 'MAR01160',
 'MAR01200',
 'MAR01201',
 'MAR01202',
 'MAR01203',
 'MAR01248',
 'MAR01249',
 'MAR01250',
 'MAR01251',
 'MAR01263',
 'MAR01264',
 'MAR01265',
 'MAR01266',
 'MAR01290',
 'MAR02310',

In [63]:
# Load in Human GEM model
model = models.init_model("HumanGEM", "homo_sapiens", 1.0, knockout=["MVK"], knockout_type="both")

In [8]:
def knockout_rxn(rxn, genes):
    """ 
    Recursively checks gene associations in given reaction to 
    determine if a reaction can be KO 
    """

    if rxn == None:
        return False 
    
    # All genes in or associations need to be KO 
    if rxn.type == "or": 
        for rxn in rxn.children:
            if not knockout_rxn(rxn, genes):
                return False 
        return True 

    elif rxn.type == "and": 
        for rxn in rxn.children: 
            if knockout_rxn(rxn, genes):
                return True 
        return False 

    elif rxn.type == "gene": 
        if rxn.gene.name in genes:
            return True
        for symbol in rxn.gene.alt_symbols: 
            if symbol in genes: 
                return True 
        return False 

In [64]:
genes = ['MVK']

In [65]:
KO_rxn_list2 = [] 

for rxn_id, rxn in model.reactions.items(): 
    if knockout_rxn(rxn.gene_associations, genes): 
        KO_rxn_list2.append(rxn_id) 

In [66]:
KO_rxn_list2

[]

In [67]:
for reaction in model.reactions.values():
    if "MVK" in reaction.list_genes():
        print(reaction.id)

In [60]:
# Look at only the two MVK reactions 

rxns = ['MAR01445', 'MAR01520']
for rxn in rxns:
    if model.reactions[rxn]

In [54]:
exchanges = {r_id: r for r_id, r in model.reactions.items()
                     if r.is_exchange}

In [144]:
model, KO_rxns = models.load_metabolic_model("HumanGEM", None, knockout=["MVK"], knockout_type="both", species = "homo_sapiens")

In [145]:
KO_rxns

{'MVK': ['MAR01445', 'MAR01520']}

In [146]:
KO_rxn_ids = []
for rxn in KO_rxns.values():
    KO_rxn_ids += rxn

In [125]:
print(['MAR01445', 'MAR01520'] in KO_rxns.values())

True


In [148]:
from compass.models.MetabolicModel import Reaction

In [150]:
uni_reactions = {}
for reaction in model.reactions.values():

    if reaction.is_pos_unidirectional:  # just ensure suffix and continue

        if (not reaction.id.endswith('_pos')) and \
                (not reaction.id.endswith('_neg')):
            reaction.id = reaction.id + "_pos"

        uni_reactions[reaction.id] = reaction

        continue

    # Copy the pos_reaction
    pos_reaction = Reaction(from_reaction=reaction)

    # Copy the negative reaction and Invert
    neg_reaction = Reaction(from_reaction=reaction)
    neg_reaction.invert()

    # Adjust bounds for both
    # Examples
    # Positive: Original -> Clipped
    #             0:10   -> 0:10
    #           -10:0    -> 0:0
    #             5:7    -> 5:7
    #            -9:-5   -> 0:0
    #
    # Negative: Original -> Flipped -> Clipped
    #             0:10   -> -10:0   -> 0:0
    #           -10:0    ->   0:10  -> 0:10
    #             5:7    ->  -7:-5  -> 0:0
    #            -9:-5   ->   5:9   -> 5:9

    if pos_reaction.upper_bound < 0:
        pos_reaction.upper_bound = 0

    if pos_reaction.lower_bound < 0:
        pos_reaction.lower_bound = 0

    if neg_reaction.upper_bound < 0:
        neg_reaction.upper_bound = 0

    if neg_reaction.lower_bound < 0:
        neg_reaction.lower_bound = 0

    pos_reaction.id = pos_reaction.id + "_pos"
    neg_reaction.id = neg_reaction.id + "_neg"

    # Only add reactions if they can carry flux
    # Checks for KO reactions 
    if pos_reaction.upper_bound > 0 or pos_reaction.id[:-4] in KO_rxn_ids:
        neg_reaction.reverse_reaction = pos_reaction
        uni_reactions[pos_reaction.id] = pos_reaction

    if neg_reaction.upper_bound > 0 or neg_reaction.id[:-4] in KO_rxn_ids:
        pos_reaction.reverse_reaction = neg_reaction
        uni_reactions[neg_reaction.id] = neg_reaction


In [151]:
# Should be around 19185 rxns
len(uni_reactions)

19189

In [152]:
model.make_unidirectional()

In [153]:
# Returns all genes in the model
genes = list(set.union(*[set(reaction.list_genes()) for reaction in uni_reactions.values()]))

In [154]:
len(genes)

7686

In [156]:
# error check in Compass code if user specifies a gene not present in the model
if "MVK" not in genes:
    print("hi")

In [155]:
print("MVK" in genes)

True


In [56]:
for rxn in KO_rxn_list2:
    model.reactions[rxn].upper_bound = 0
    model.reactions[rxn].lower_bound = 0

In [32]:

model.reactions['MAR03905'].gene_associations.children

In [5]:
# Observe the expression data makeup
data_path = ['/home/dzhang/Compass_Results/PANC_scRNAseq/compass_files/ASPC1/DTP/ASPC1_matrix.mtx', 
        '/home/dzhang/Compass_Results/PANC_scRNAseq/compass_files/ASPC1/DTP/ASPC1_genes.tsv',
        '/home/dzhang/Compass_Results/PANC_scRNAseq/compass_files/ASPC1/DTP/ASPC1_barcodes.tsv']

expression = utils.read_data(data_path)

In [6]:
expression

,TCAGCTCTCAGCATGT-2,CATCGGGAGCACCGCT-2,TACGGGCAGTTAGGTA-2,GATCGATGTAGGAGTC-2,ACCTTTACAGCGATCC-2,CTCTAATGTCCGACGT-2,ATTATCCGTCTAGTGT-2,GAAATGAAGTGTACTC-2,TCACAAGAGGTGTGGT-2,TTGTAGGAGAGGGCTT-2,...,TATCAGGCAGGAATCG-2,CGCTGGAGTAACGCGA-2,AGATTGCCAAGAGGCT-2,TGACGGCAGTCATCCA-2,CATCAAGCATTACGAC-2,GGACAAGTCCAAACAC-2,AGCGTATGTTCCACAA-2,TCTCATAAGTTCCACA-2,AAGCCGCAGCCAACAG-2,ACGATACTCGTACCGG-2
WASH7P,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
RP11-34P13.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
CICP27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
RP11-34P13.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
RP11-34P13.13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC004556.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
AC136352.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
AC136616.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
AC007325.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.2833,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
genes = ['CDIPT',
 'CMPK1',
 'CYP51A1',
 'DHCR24',
 'EBP',
 'FDFT1',
 'FOSL1',
 'HMGCR',
 'HMGCS1',
 'HSD17B7',
 'LSS',
 'MVD',
 'MVK',
 'NDUFA4',
 'NSDHL',
 'PGS1',
 'PMVK',
 'PTPMT1',
 'SC5D',
 'SLC29A1',
 'SQLE',
 'UGDH']

In [80]:
# Testing KO of fluxes 
model = models.load_metabolic_model("HumanGEM", None, genes)

In [12]:
for rxn in KO_rxn_list2: 
    print(model.reactions[rxn].upper_bound)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [13]:
# Test knockout of models (i.e. when the bounds are set to 0, is our r_max 0?) 
problem = compass.algorithm.initialize_cplex_problem(model)

In [15]:
for rxn in KO_rxn_list2:
    reaction = model.reactions[rxn]
    if reaction.is_exchange:
        continue

    partner_reaction = reaction.reverse_reaction

    # Set partner reaction upper-limit to 0 in problem
    # Store old limit for later to restore
    if partner_reaction is not None:
        partner_id = partner_reaction.id
        old_partner_ub = problem.variables.get_upper_bounds(partner_id)
        old_partner_lb = problem.variables.get_lower_bounds(partner_id)
        problem.variables.set_upper_bounds(partner_id, max(old_partner_lb, 0))
    
    utils.reset_objective(problem)
    problem.objective.set_linear(rxn, 1.0)
    problem.objective.set_name(str(rxn))
    problem.objective.set_sense(problem.objective.sense.maximize)

    problem.solve()
    rxn_max = problem.solution.get_objective_value()

    print(rxn_max) 

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
